AUTHOR: SPARSHSAH

# BAB: The Intoxicating Enticement of Levered Returns

I'm going to keep things simple here for illustration. But

* Excess-of-financing-rate returns (the right way to think about returns),
* MVO (mean-variance optimation, the core system here),
* BAB (betting against beta, the driver of the benefit of leverage),
* The fact that in markets we see a 100-year flood about once every 10 years (sadface), and
* Beta compression (the villain of this story, which prevents us from eating a free lunch)

are extremely well-developed concepts in quantitative finance (both stock selection and global macro), and you really ought to study them for a fuller picture.

## Givens

* You have $\$100$ to invest.
* You can choose between two assets, each priced at $\$100$ right now:
    * Asset A, with ER 20% and vol 10% (and 100-year-flood return -30%).
    * Asset B, with ER 12% and vol 4% (and 100-year-flood return -20%).
    * You may not split your allocation between the two -- That is diversification, which is another powerful concept but not the point here.
* Cost of leverage is 3% per year.

## Calculations

Let's set up some calculations, assuming that the center of the distribution of asset returns is Normal, so that your 95% return-prediction interval is the mean $\pm$ 2 (really 1.96, but ok) std's.

* Asset A's expected price at year-end is $\$100 + \$20 = \$120$.
    * Its 95% price-prediction interval is $\$100 + (\$20 \pm 2 \cdot \$10) = [\$100,\, \$140]$.
    * Its 100-year-flood price is $\$100 - \$30 = \$70$.
* Asset B's expected price at year-end is $\$100 + \$12 = \$112$.
    * Its 95% price-prediction interval is $\$100 + (\$12 \pm 2 \cdot \$4) = [\$104,\, \$120]$.
    * Its 100-year-flood price is $\$100 - \$20 = \$80$.

## Strategy

* Asset A's expected return is much higher than Asset B's (you expect to end up with $\$120$ at the end of the year instead of $\$112$), but
* Asset B is safer than Asset A (your "bear" price is $\$104$ instead of $\$100$, and your "100-year-flood" price is $\$80$ instead of $\$70$).

Which should you choose?

### A third alternative

Well, let's consider a third alternative. In this strategy, you
* Take your $\$100$ and invest it in Asset B.
* Then, you go to the bank, _borrow another $\$100$_, and invest _that_ in B.
* You now have $\$200$ invested in B (assets) and a $\$100$ loan from the bank (liabilities).
    * Some people call this "one turn of leverage".
    * Other call it "1:1" debt:equity ratio.
    * The right way to call it is double- (or 2x-) levered.

#### Calculations

What is the ex-ante distribution of outcomes here?
* You expect that B will end the year at $\$112$, so that your position will be worth $\$224$. You must then repay the bank their $\$100$, plus their $\$3$ of interest. So, you will end the year with $\$224 - \$100 - \$3 = \$121$.
* In the bear case, B will end the year at $\$104$, so that your position will be worth $\$208$. After you've repaid the bank their $\$103$, you'll end the year with $\$208 - \$103 = \$105$.
* In the bull case, B will end the year at $\$120$, your position will be worth $\$240$, and once you've repaid the bank you'll end with $\$240 - \$103 = \$137$.
* In case of a 100-year flood, B will end the year at $\$80$, your position will be worth $\$160$, and once you've repaid the bank you'll end with $\$160 - \$103 = \$57$.

#### Commentary

I've set these numbers to be actually realistic, so that there is a genuine tradeoff. (If there weren't, the anomalous profits to this strategy would probably be quickly arb'ed away -- No pain, no gain. The fact that factors can experience drawdowns that make them hard to stick with is ironically probably a desirable feature, as it scares people away from arb'ing away the mispricing.)

* In expectation, the third alternative ends at $\$121$, higher than A's $\$120$ and much higher than B's $\$112$.
* In the bear case, the third alternative ends at $\$105$, much higher than A's $\$100$ and also higher than B's $\$104$.
* In the bull case, the third alternative ends at $\$137$, not quite as high as A's $\$140$ but much higher than B's $\$120$.
* In a 100-year flood, the third alternative ends at $\$57$, much lower than A's $\$70$ and much _much_ lower than B's $\$80$.

So, what would you choose? A, B, or juiced B? If you're on the fence, what if you redo the exercise but assume taking only 1.5x leverage in B? This will kill your ER advantage over A, but you'll still have a meaningful ER advantage over unlevered B, and will still be much safer than A.

# Asymmetry: The Vexatious Nature of Levered Returns

First of all, note that maintaining leverage over some given horizon requires you to dynamically rebalance to target.

For example, suppose SPXL, Direxion's Daily 3x-Levered Bull SPX ETF, has USD1 NAV (USD1 client assets + USD2 borrowed money - USD2 owed on the loan), and hence invests USD3 in the SP500. Then suppose today the SP500 return 1pct (assume zero interest rates), growing SPXL's NAV to USD1.03.

This means Direxion must go back to the bank and borrow an additional USD0.06, because now (assuming they didn't buy or sell any of the shares, meaning that conveniently their holdings naturally evolved to reflect the new composition of the SP500, which is a convenient feature of market-cap-weighted benchmarks and indeed a cornerstone of truly "passive" investment strategies) they are technically at 2.94x=USD3.03/USD1.03 leverage which is slightly less than their 3x stated target.

Second of all, take a look at the below. Levering up to 1.20x seems like a good tradeoff: If you consistently win, you overall win fully 1.473x as much; whereas if you consistently lose, you overall lose only 1.068x as much. But levering down to 0.80x seems like a bad tradeoff: If you consistently win, you overall win only 0.657x as much; whereas if you consistently lose, you overall lose fully 0.905x as much. Note that the "contrapositive" therefore also holds: Based on this logic, levering up from 0.80x to 1.00x seems like a good tradeoff. The upshot is that deleveraging as a risk-reduction tactic might dampen your wins more than expected, while simultaneously dampening your losses less than expected.

For a more optimistic example of a similar financial "convexity" phenomenon, cf Aaron Brown at https://www.bloomberg.com/opinion/articles/2020-05-26/twitter-spat-over-market-risks-is-a-teaching-moment.

In [1]:
ANN = 261  # financial calendar daycount (# of trading days per year)
def cmp_pnl(lev=1, ltv=0.10, ann=ANN, sign=+1, std=1, dur=ANN):
    """
    Compare `lev`x-levered vs 1x-levered pnl
    if we realize `sign*std`STD pnl
    (at `ltv` annualized long-term vol)
    for `dur`days in a row.
    """
    def get_pnl(lev):
        print ("\tleverage = {lev}x".format(lev=lev))
        daily_pnl = lev * sign * std * ltv / ann**0.5
        px = (1 + daily_pnl)**dur
        pnl = px - 1
        print("\t\tfinal price index = ${px}".format(px=round(px, 2)))
        print("\t\tfinal pnl = {pnl}%".format(pnl=round(100*pnl, 1)))
        return pnl
    unlev_pnl = get_pnl(lev=1)
    lev_pnl = get_pnl(lev=lev)
    print("\t>>>> ratio = {} <<<<".format(round(lev_pnl / unlev_pnl, 3)))

for lev in 1.20, 0.80:
    for sign in +1, -1:
        print("SIGN = {sign}".format(sign=sign))
        cmp_pnl(lev=lev, sign=sign)
    print()

SIGN = 1
	leverage = 1x
		final price index = $5.01
		final pnl = 400.6%
	leverage = 1.2x
		final price index = $6.9
		final pnl = 590.0%
	>>>> ratio = 1.473 <<<<
SIGN = -1
	leverage = 1x
		final price index = $0.2
		final pnl = -80.2%
	leverage = 1.2x
		final price index = $0.14
		final pnl = -85.7%
	>>>> ratio = 1.068 <<<<

SIGN = 1
	leverage = 1x
		final price index = $5.01
		final pnl = 400.6%
	leverage = 0.8x
		final price index = $3.63
		final pnl = 263.0%
	>>>> ratio = 0.657 <<<<
SIGN = -1
	leverage = 1x
		final price index = $0.2
		final pnl = -80.2%
	leverage = 0.8x
		final price index = $0.27
		final pnl = -72.6%
	>>>> ratio = 0.905 <<<<

